The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [2]:
import requests
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: deprecated parameters","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","Feb

Extract table from html

In [4]:
temp_table = soup.find('table',{'class':'wikitable sortable'})
temp_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Extract rows from the table

In [5]:
output_rows = []
for table_row in temp_table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)

output_rows

[[],
 ['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n'],
 ['M5A', 'Downtown Toronto', 'Harbourfront\n'],
 ['M5A', 'Downtown Toronto', 'Regent Park\n'],
 ['M6A', 'North York', 'Lawrence Heights\n'],
 ['M6A', 'North York', 'Lawrence Manor\n'],
 ['M7A', "Queen's Park", 'Not assigned\n'],
 ['M8A', 'Not assigned', 'Not assigned\n'],
 ['M9A', 'Etobicoke', 'Islington Avenue\n'],
 ['M1B', 'Scarborough', 'Rouge\n'],
 ['M1B', 'Scarborough', 'Malvern\n'],
 ['M2B', 'Not assigned', 'Not assigned\n'],
 ['M3B', 'North York', 'Don Mills North\n'],
 ['M4B', 'East York', 'Woodbine Gardens\n'],
 ['M4B', 'East York', 'Parkview Hill\n'],
 ['M5B', 'Downtown Toronto', 'Ryerson\n'],
 ['M5B', 'Downtown Toronto', 'Garden District\n'],
 ['M6B', 'North York', 'Glencairn\n'],
 ['M7B', 'Not assigned', 'Not assigned\n'],
 ['M8B', 'Not assigned', 'Not assigned\n'],
 ['M9B', 'Etobicoke', 'Cloverdale

Create Dataframe

In [6]:
df_Toronto = pd.DataFrame(output_rows)

df_Toronto.columns = ['PostalCode','Borough','Neighbourhood']

df_Toronto.head()

,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [7]:
df_Toronto.drop(index=0,inplace=True)

In [8]:
df_Toronto.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


In [9]:
for rows in df_Toronto:
    df_Toronto['Neighbourhood'] = df_Toronto['Neighbourhood'].replace('\n', '', regex=True)


Remove rows with Not assigned as Borough

In [10]:
df_Toronto = df_Toronto[df_Toronto['Borough'] != 'Not assigned']
df_Toronto

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [11]:
df_Toronto.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Group the neighbourhoods with the same postal code and borough

In [12]:
df_Toronto_clean = (df_Toronto.groupby('PostalCode').agg({'Borough' : 'first', 'Neighbourhood' : ','.join}).reset_index().reindex(columns=df_Toronto.columns))
df_Toronto_clean

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [13]:
import numpy as np

Replace the neighbourhoods that are Not assigned with the name of the Borough

In [14]:
df_Toronto_clean['Neighbourhood'] = np.where(df_Toronto_clean['Neighbourhood'] == 'Not assigned', df_Toronto_clean['Borough'], df_Toronto_clean['Neighbourhood'])
df_Toronto_clean

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [15]:
df_Toronto_clean.shape

(103, 3)

Using the CSV file on the link

In [16]:
df_geocodes = pd.read_csv('https://cocl.us/Geospatial_data')
df_geocodes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Merging the 2 dataframes based on postal code 

In [60]:
df_merged = pd.merge(left=df_Toronto_clean,right=df_geocodes, left_on='PostalCode', right_on='Postal Code')
df_merged

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


In [61]:
df_merged.drop(columns='Postal Code', inplace = True)

In [62]:
df_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [63]:
df_merged.shape

(103, 5)

In [64]:
df_merged['Has_Toronto_in_Borough_name'] = np.where(df_merged.Borough.str.contains("Toronto"),1,0)
df_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Has_Toronto_in_Borough_name
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0


In [65]:
# set number of clusters
k_clusters = 6

toronto_grouped_clustering = df_merged.drop(['PostalCode','Borough','Neighbourhood'], axis = 1)

In [66]:
toronto_grouped_clustering

,Latitude,Longitude,Has_Toronto_in_Borough_name
0,43.806686,-79.194353,0
1,43.784535,-79.160497,0
2,43.763573,-79.188711,0
3,43.770992,-79.216917,0
4,43.773136,-79.239476,0
5,43.744734,-79.239476,0
6,43.727929,-79.262029,0
7,43.711112,-79.284577,0
8,43.716316,-79.239476,0
9,43.692657,-79.264848,0


In [67]:
# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 4, 2, 4], dtype=int32)

In [68]:
# add clustering labels
df_merged.insert(0, 'Cluster Labels', kmeans.labels_)

In [69]:
toronto_merged = df_merged
toronto_merged

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Has_Toronto_in_Borough_name
0,2,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0
1,2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0
3,2,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0
5,2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0
6,2,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0
7,4,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0
8,2,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0
9,4,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0


In [70]:
df_merged.head()

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Has_Toronto_in_Borough_name
0,2,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0
1,2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0
3,2,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0


In [71]:
# create map
toronto_latitude = 43.65
toronto_longitude = -79.38
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters